In [1]:
from pycaret.datasets import get_data
from pycaret.classification import *
import pandas as pd
import os

In [2]:
pd.set_option('display.max_rows', None, 'display.max_columns', None)

In [3]:
file_path = os.path.join(os.getcwd(), '..', 'Datasets', 'CSE-CIC-IDS2018', 'pre-processed','amostra_5_0_or_1.csv')

In [4]:
data = pd.read_csv(file_path)

In [5]:
# Setup the PyCaret environment
autoML = ClassificationExperiment()
autoML.setup(data,
        target = 'Label',
        preprocess=True,
        train_size=0.1,
        fix_imbalance=True,
        fold=3,
        normalize=True,
        remove_outliers=True
        )

KeyboardInterrupt: 

In [6]:
# Compare models
best_model = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
qda,Quadratic Discriminant Analysis,0.9909,0.0000,0.9909,0.9932,0.9909,0.9864,0.9875,0.0160
lda,Linear Discriminant Analysis,0.9809,0.0000,0.9809,0.9857,0.9809,0.9715,0.9739,0.0140
lr,Logistic Regression,0.9718,0.0000,0.9718,0.9789,0.9718,0.9579,0.9614,0.5870
nb,Naive Bayes,0.9709,0.0000,0.9709,0.9782,0.9709,0.9566,0.9602,0.0110
dt,Decision Tree Classifier,0.9709,0.0000,0.9709,0.9782,0.9709,0.9566,0.9602,0.0120
rf,Random Forest Classifier,0.9709,0.0000,0.9709,0.9782,0.9709,0.9566,0.9602,0.0520
gbc,Gradient Boosting Classifier,0.9709,0.0000,0.9709,0.9782,0.9709,0.9566,0.9602,0.1150
knn,K Neighbors Classifier,0.9627,0.0000,0.9627,0.9716,0.9623,0.9440,0.9486,0.0190
ada,Ada Boost Classifier,0.9618,0.0000,0.9618,0.9714,0.9618,0.9430,0.9477,0.0390
et,Extra Trees Classifier,0.9618,0.0000,0.9618,0.9714,0.9618,0.9430,0.9477,0.0630
